##Data Preparation

Read the cancer_classification.csv file into a DataFrame using pandas.
Check for any missing values and handle them appropriately.

In [17]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Phase2/Week4/Day4/Copy of cancer_classification.csv")

df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,benign_0__mal_1
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [18]:
df.isnull().sum()

,0
mean radius,0
mean texture,0
mean perimeter,0
mean area,0
mean smoothness,0
mean compactness,0
mean concavity,0
mean concave points,0
mean symmetry,0
mean fractal dimension,0


##Preprocess the Data

Split the dataset into features (X) and target (y).
Split the data into training and testing sets using train_test_split.

In [19]:
y=df.benign_0__mal_1
X=df.drop('benign_0__mal_1',axis=1)


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.20,random_state=42,stratify=y) #


##Define the Objective Function

Create an objective function for Optuna that:
Suggests hyperparameters for a model (e.g., GradientBoostingClassifier).
Trains the model on the training set.
Evaluates the model on the testing set.
Returns the accuracy.

In [20]:
import optuna
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 7),  # int
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 10, 20),  # int
        'splitter': trial.suggest_categorical('splitter', ['best', 'random']),  # categorical
        'min_weight_fraction_leaf': trial.suggest_float('min_weight_fraction_leaf', 0.0, 0.5),  # float
        'random_state': trial.suggest_int('random_state', 10, 60),  # int
    }

    model = DecisionTreeClassifier(**params)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)

    return accuracy

##Set Up Optuna Study

Create an Optuna study to optimize the objective function.
Run the optimization for a specified number of trials (e.g., 100).

In [21]:

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-06-05 10:34:18,420] A new study created in memory with name: no-name-40dfb374-d928-4a15-b296-7e6b6872e392
[I 2025-06-05 10:34:18,438] Trial 0 finished with value: 0.8771929824561403 and parameters: {'max_depth': 6, 'max_leaf_nodes': 16, 'splitter': 'random', 'min_weight_fraction_leaf': 0.09268614752945437, 'random_state': 20}. Best is trial 0 with value: 0.8771929824561403.
[I 2025-06-05 10:34:18,451] Trial 1 finished with value: 0.9035087719298246 and parameters: {'max_depth': 3, 'max_leaf_nodes': 15, 'splitter': 'random', 'min_weight_fraction_leaf': 0.28006215880788166, 'random_state': 21}. Best is trial 1 with value: 0.9035087719298246.
[I 2025-06-05 10:34:18,464] Trial 2 finished with value: 0.8947368421052632 and parameters: {'max_depth': 4, 'max_leaf_nodes': 13, 'splitter': 'best', 'min_weight_fraction_leaf': 0.3958632301581101, 'random_state': 33}. Best is trial 1 with value: 0.9035087719298246.
[I 2025-06-05 10:34:18,477] Trial 3 finished with value: 0.9122807017543859 

##Retrieve and Display Results

Access the best hyperparameters found during the optimization.
Train a final model using these best parameters on the entire training data.
Evaluate this model on the test dataset and display the accuracy.

In [22]:
best_params = study.best_params
print("Best Parameters:", best_params)

# Access the best score
best_score = study.best_value
print("Best Score:", best_score)

Best Parameters: {'max_depth': 7, 'max_leaf_nodes': 12, 'splitter': 'best', 'min_weight_fraction_leaf': 0.017549321693810543, 'random_state': 57}
Best Score: 0.9473684210526315


In [23]:
final_model = DecisionTreeClassifier(**best_params)
final_model.fit(X_train, y_train)


y_test_pred = final_model.predict(X_test)
final_accuracy = accuracy_score(y_test, y_test_pred)
print("Final Model Accuracy on Test Data:", final_accuracy)

Final Model Accuracy on Test Data: 0.9473684210526315
